In [21]:
import sys
import html
import pandas
import pickle
import json
sys.path.append('..')
from aips import *

import warnings
warnings.filterwarnings('ignore') #Some operations warn inside a loop, we'll only need to see the first warning

In [2]:
outdoors_collection="outdoors"
path = "./"

/opt/anaconda3/envs/sbert/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
def getQuestions():
    qtypes = ['Who','What','When','Where','Why','How']
    questions = []
    for qt in qtypes:
        lq = len(qt)
        request = {
            "query": qt,
            "fields": ["id", "url", "owner_user_id", "title", "accepted_answer_id"],
            "params": {
              "qf": ["title"],
              "fq": ["accepted_answer_id:[* TO *]"],
              "defType": "edismax",
              "rows":10000
            }
        }
        docs = requests.post(solr_url + outdoors_collection + "/select", json=request).json()["response"]["docs"]
        questions += [doc for doc in docs if doc['title'][0:lq]==qt]             
    return questions

/opt/anaconda3/envs/sbert/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
def getContextDataFrame(questions):
    contexts={"id":[],"question":[],"context":[],"url":[]}
    for question in questions:
        request = {
            "query": "*:*",
            "fields": ["body"],
            "params": {
              "fq": ["id:"+str(question["accepted_answer_id"])],
              "defType": "edismax",
              "rows":1,
              "sort":"score desc"
            }
        }
        docs = requests.post(solr_url + outdoors_collection + "/select", json=request).json()["response"]["docs"]
        contexts["id"].append(question["id"])
        contexts["url"].append(question["url"])
        contexts["question"].append(question["title"]),
        contexts["context"].append(docs[0]["body"])
    return pandas.DataFrame(contexts)

In [15]:
questions = getQuestions()
contexts = getContextDataFrame(questions)
contexts[0:10]

,id,question,context,url
0,4410,Who places the anchors that rock climbers use?,There are two distinct styles of free rock cli...,https://outdoors.stackexchange.com/questions/4410
1,5347,"Who places the bolts on rock climbing routes, ...",What you're talking about is Sport climbing. G...,https://outdoors.stackexchange.com/questions/5347
2,20662,Who gets the bill if you activate a PLB to hel...,"Almost always the victim gets the bill, but as...",https://outdoors.stackexchange.com/questions/2...
3,7623,What knot is this one? What are its purposes?,Slip knot It's undoubtably a slip knot that's ...,https://outdoors.stackexchange.com/questions/7623
4,11587,"What sort of crane, and what sort of snake?","To answer the snake part of it, looking at som...",https://outdoors.stackexchange.com/questions/1...
5,1660,What is Geocaching?,"In short, it's a high-tech treasure hunt. geoc...",https://outdoors.stackexchange.com/questions/1660
6,913,What is a buff?,To be honest I was dubious about getting somet...,https://outdoors.stackexchange.com/questions/913
7,4904,What Rope to purchase?,"Short answer: For your first rope, none of the...",https://outdoors.stackexchange.com/questions/4904
8,6397,What is a bloquers?,"I can only assume, that it derives from bloque...",https://outdoors.stackexchange.com/questions/6397
9,10173,What is a longbow?,The problem with the longbow discussion is tha...,https://outdoors.stackexchange.com/questions/1...


In [22]:
#contexts.to_csv(path+'data/question-answer-seed-contexts.csv',index=False)
contexts = pandas.read_csv(path+'data/question-answer-seed-contexts.csv')

In [23]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import tqdm

model_name = 'deepset/roberta-base-squad2'
pipeline_type = 'question-answering'

device=-1 #CPU
#device=0 #<-- Uncomment to use GPU, if you are running in Google Colab

def answerQuestions(contexts,k=10):
    nlp = pipeline(pipeline_type, model=model_name, tokenizer=model_name, device=device)
    guesses = []
    for idx,row in tqdm.tqdm(contexts[0:k].iterrows(),total=k):
        result = nlp({"question":row["question"],"context":row["context"]})
        guesses.append(result)
    return guesses

In [24]:
guesses = answerQuestions(contexts,k=len(contexts))
contexts["answers"] = guesses
print(guesses[0:10])
"""
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1662/1662 [43:19<00:00,  1.56s/it] 

[{'score': 0.2776225209236145, 'start': 474, 'end': 517, 'answer': 'a local enthusiast or group of enthusiasts.'}, {'score': 0.1954791247844696, 'start': 81, 'end': 118, 'answer': 'the person who is creating the climb.'}, {'score': 0.024139929562807083, 'start': 14, 'end': 24, 'answer': 'the victim'}, {'score': 0.3299234211444855, 'start': 29, 'end': 38, 'answer': 'slip knot'}, {'score': 0.0005422658286988735, 'start': 1255, 'end': 1262, 'answer': 'aquatic'}, {'score': 0.37733304500579834, 'start': 15, 'end': 41, 'answer': 'a high-tech treasure hunt.'}, {'score': 0.5653401613235474, 'start': 192, 'end': 233, 'answer': 'a tube of lightweight, stretchy material.'}, {'score': 0.10057392716407776, 'start': 125, 'end': 155, 'answer': 'the cheapest one of the three,'}, {'score': 0.7781326174736023, 'start': 68, 'end': 77, 'answer': 'blocking.'}, {'score': 0.2520507276058197, 'start': 227, 'end': 266, 'answer': 'the traditional longbow made from wood,'}]
"""

Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1662/1662 [43:19<00:00,  1.56s/it] 

[{'score': 0.2776225209236145, 'start': 474, 'end': 517, 'answer': 'a local enthusiast or group of enthusiasts.'}, {'score': 0.1954791247844696, 'start': 81, 'end': 118, 'answer': 'the person who is creating the climb.'}, {'score': 0.024139929562807083, 'start': 14, 'end': 24, 'answer': 'the victim'}, {'score': 0.3299234211444855, 'start': 29, 'end': 38, 'answer': 'slip knot'}, {'score': 0.0005422658286988735, 'start': 1255, 'end': 1262, 'answer': 'aquatic'}, {'score': 0.37733304500579834, 'start': 15, 'end': 41, 'answer': 'a high-tech treasure hunt.'}, {'score': 0.5653401613235474, 'start': 192, 'end': 233, 'answer': 'a tube of lightweight, stretchy material.'}, {'score': 0.10057392716407776, 'start': 125, 'end': 155, 'answer': 'the cheapest one of the three,'}, {'score': 0.7781326174736023, 'start': 68, 'end': 77, 'answer': 'blocking.'}, {'score': 0.2520507276058197, 'start': 227, 'end': 266, 'answer': 'the traditional longbow made from wood,'}]


In [25]:
contexts.to_csv(path+'data/question-answer-squad2-guesses.csv',index=False)

In [23]:
"""
with open('data/guesses_all.json','r') as fd:
    guesses_all = json.load(fd)
print(len(guesses_all))
print(json.dumps(guesses_all[0],indent=2))
"""

1662
{
  "question": "Who places the anchors that rock climbers use?",
  "context": "There are two distinct styles of free rock climbing (with, as always, some grey areas). In \"trad\" (= traditional) climbing, the team carries their own removable gear. The leader places protection points whilst climbing and the second climber removes them again. There might be the odd piece of permanent gear - pitons or bolts - but the climbing team is basically responsible for arranging their own protection. In sport climbing, bolts are pre-placed by somebody , usually a local enthusiast or group of enthusiasts. How stuff is funded/maintained varies a lot from area to area, but it is almost always voluntary efforts of some kind. Formal standards don't exist, although most areas have best practices that are generally understood and adhered to by people putting in or maintaining fixed equipment. Exactly what these best practices are vary a lot from area to area depending on type of rock, proximity to s

In [28]:
"""
def guessesToDataFrame(guesses):
    table={
        "id":[],
        "question":[],
        "context":[],
        #"result":[],
        "score":[],
        "start":[],
        "end":[],
        "answer":[]
    }
    i=0
    for guess in guesses:
        table["id"].append(i)
        table["question"].append(guess["question"])
        table["context"].append(guess["context"])
        #table["result"].append(guess["result"])
        table["score"].append(guess["result"]["score"])
        table["start"].append(guess["result"]["start"])
        table["end"].append(guess["result"]["end"])
        table["answer"].append(guess["result"]["answer"])
        i+=1

    return pandas.DataFrame(table)
guesses_df = guessesToDataFrame(guesses_all)
guesses_df
"""

,id,question,context,score,start,end,answer
0,0,Who places the anchors that rock climbers use?,There are two distinct styles of free rock cli...,0.322016,474,516,a local enthusiast or group of enthusiasts
1,1,"Who places the bolts on rock climbing routes, ...",What you're talking about is Sport climbing. G...,0.217747,81,117,the person who is creating the climb
2,2,Who gets the bill if you activate a PLB to hel...,"Almost always the victim gets the bill, but as...",0.518512,2347,2357,the victim
3,3,What knot is this one? What are its purposes?,Slip knot It's undoubtably a slip knot that's ...,0.229166,29,38,slip knot
4,4,"What sort of crane, and what sort of snake?","To answer the snake part of it, looking at som...",0.124369,1255,1262,aquatic
...,...,...,...,...,...,...,...
1657,1657,How much margin due to charging inefficiency s...,The overall efficiency from a power bank to a ...,0.490509,887,890,50%
1658,1658,How do you dry wet socks when camping/backpack...,Sleeping with the socks on your torso is the m...,0.063256,0,37,Sleeping with the socks on your torso
1659,1659,How do simple compass needles remain level ins...,The short answer is that the manufacturers kno...,0.013619,223,262,balancing a compass for a specific zone
1660,1660,How do I apply for a permit for the Zona de Re...,The phone number is right: + 34 956 70 97 33 (...,0.182854,99,112,send an email


In [30]:
#guesses_df.to_csv('data/guesses.csv')

# Training!

In [4]:
import datasets

def getTrainingData(filename):
    golden_answers = pandas.read_csv(filename)
    golden_answers["class"] = golden_answers["class"].fillna(-2).astype(int)
    validated=golden_answers[golden_answers["class"]>-1]
    
    table={
        "id":[],
        "url":[],
        "title":[],
        "question":[],
        "context":[],
        "answers":[],
        "owner_user_id":[]
    }
    
    for idx,row in validated.iterrows():
        answers = row["gold"].split('|')
        starts = []
        notfound = False
        for i in range(len(answers)):
            found = row["context"].find(answers[i])
            starts.append(found)
            if(found<0):
                notfound = True
        if not notfound:
            table["id"].append(row["id"])
            table["url"].append(row["url"])
            table["title"].append(row["question"])
            table["question"].append(row["question"])
            table["context"].append(row["context"])
            table["answers"].append({
                "text":answers,
                "answer_start":starts
            })
            table["owner_user_id"].append(row["owner_user_id"])
    df = pandas.DataFrame(table).sample(frac=1)
    train_split = int(len(df)*0.8)
    train_dataset = datasets.Dataset.from_pandas(df[:train_split])
    test_dataset = datasets.Dataset.from_pandas(df[train_split:])
    datadict = datasets.DatasetDict({'train':train_dataset,'test':test_dataset})
    return datadict

#This golden answers file was labeled by me (Max Irwin).
#It took about 2-3 hours to label 200 question/answer rows
#Doing so will give you a deeper appreciation for the difficulty of the NLP task.
#I *highly* encourage you to label even more documents, and re-run the fine-tuning tasks coming up.
datadict = getTrainingData(path+'data/outdoors_golden_answers.csv')

/opt/anaconda3/envs/sbert/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
training_data

/opt/anaconda3/envs/sbert/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dataset({
    features: ['id', 'title', 'question', 'context', 'answers', '__index_level_0__'],
    num_rows: 86
})

In [20]:
import transformers
tokenizer = transformers.RobertaTokenizerFast.from_pretrained('roberta-base')
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [23]:
example = training_data[0]
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=384,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=127
)

assert isinstance(tokenized_example, transformers.BatchEncoding)

print(tokenized_example["offset_mapping"][0][:100])
sequence_ids = tokenized_example.sequence_ids(1)
print(sequence_ids)

[(0, 0), (0, 4), (5, 10), (11, 17), (18, 19), (20, 25), (26, 30), (31, 35), (35, 42), (42, 43), (0, 0), (0, 0), (0, 1), (2, 5), (6, 8), (9, 14), (15, 18), (19, 28), (28, 29), (30, 33), (33, 38), (39, 44), (45, 46), (46, 47), (47, 48), (48, 49), (49, 51), (52, 56), (57, 61), (62, 65), (66, 70), (71, 73), (74, 79), (80, 82), (83, 85), (85, 87), (88, 91), (92, 101), (102, 104), (105, 110), (111, 117), (118, 121), (122, 132), (132, 133), (134, 139), (140, 143), (144, 149), (150, 153), (154, 160), (161, 163), (164, 166), (167, 174), (174, 175), (176, 179), (179, 180), (180, 184), (185, 190), (190, 195), (196, 200), (200, 207), (208, 212), (212, 213), (214, 218), (218, 224), (225, 228), (229, 231), (231, 234), (234, 236), (236, 241), (241, 242), (243, 246), (247, 250), (251, 256), (257, 259), (260, 264), (265, 270), (271, 273), (274, 277), (278, 282), (282, 289), (290, 297), (297, 299), (300, 306), (307, 313), (314, 317), (318, 321), (322, 327), (328, 333), (334, 340), (340, 341), (342, 345)

AttributeError: 

In [13]:
def prepare_dataset(examples):

    maximum_tokens = 384 # This will be the number of tokens in BOTH the question and context
    document_overlap = 128 # Sometimes we need to split the context into smaller chunks, so we will overlap with this window
    pad_on_right = tokenizer.padding_side == "right"
    
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=maximum_tokens,
        stride=document_overlap,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )
    
    print(tokenized_examples[0])

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

train_dataset = prepare_dataset(training_data[:1])

Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


AttributeError: 

In [79]:
from transformers import RobertaForQuestionAnswering, Trainer, TrainingArguments
import torch

model = RobertaForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')

training_args = TrainingArguments(
    output_dir='data/questionanswering/results',          # output directory
    num_train_epochs=3,                                   # total # of training epochs
    per_device_train_batch_size=16,                       # batch size per device during training
    per_device_eval_batch_size=64,                        # batch size for evaluation
    warmup_steps=500,                                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                                    # strength of weight decay
    logging_dir='data/questionanswering/logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)


#question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
#inputs = tokenizer(question, text, return_tensors='pt')
#start_positions = torch.tensor([1])
#end_positions = torch.tensor([3])

#outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
#loss = outputs.loss
#start_scores = outputs.start_logits
#end_scores = outputs.end_logits

In [80]:
trainer.train()

ValueError: You have to specify either input_ids or inputs_embeds